In [2]:
from __future__ import print_function, division, absolute_import
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))
# from clr_callback import CyclicLR
# import dill
from BalancedDannAudioDataGenerator import BalancedAudioDataGenerator
import os,time
from scipy.io import loadmat
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
import math
import pandas as pd
import tables,h5py
from datetime import datetime
import argparse
from keras.callbacks import TensorBoard, Callback, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger
from keras import backend as K
from keras.utils import plot_model
from Heartnet import heartnet,getAttentionModel
from utils import log_macc, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import Evaluator
import dataLoader
from custom_layers import Attention
from Gradient_Reverse_Layer import GradientReversal
sns.set()
from keras.models import Model

In [3]:
num_filt = (8, 4)
num_dense = 20
lr = 0.001
bn_momentum = 0.99
eps = 1.1e-5
bias = False
l2_reg = 0.04864911065093751
l2_reg_dense = 0.
kernel_size = 5
maxnorm = 10000.
dropout_rate = 0.5
dropout_rate_dense = 0.
padding = 'valid'
activation_function = 'relu'
subsam = 2
FIR_train= True
trainable = True
hp_lambda = np.float32(0)
lr_decay =0.0001132885
random_seed = 1
num_class =2
num_class_domain = 18
convtype= 1
decision = 'majority' 

In [4]:
loadpath = '../../Adversarial Heart Sound Results/models/bcdefghi_a 2019-10-05 11:26:55.596421/weights.0152-0.5729.hdf5'

In [5]:
model = heartnet(loadpath,activation_function, bn_momentum, bias, dropout_rate, dropout_rate_dense,
                             eps, kernel_size, l2_reg, l2_reg_dense, lr, lr_decay, maxnorm,
                             padding, random_seed, subsam, num_filt, num_dense, FIR_train, trainable, convtype,
                             num_class=num_class,num_class_domain=9,hp_lambda=hp_lambda)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
conv1d_linearphase_type_1 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_2 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_3 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_lin

In [19]:
model.optimizer.lr

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [7]:
model = getAttentionModel(model,'bcdefghi_a',lr,lr_decay)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
conv1d_linearphase_type_1 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_2 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_3 (Conv (None, 2500, 1)      30          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_lin

In [31]:
layers = {x.name:x for x in model.layers[-5:]}
layers
while('flatten' not in model.layers[-1].name):
    model.layers.pop()

In [33]:
for layer in model.layers:
    layer.trainable = True

In [32]:
merged = Attention(name='att')(model.layers[-1].output)
dann_in = layers['grl'](merged)
dsc = layers['domain_dense'](dann_in)
dsc = layers['domain'](dsc)
merged = layers['class_dense'](merged)
merged = layers['class'](merged)
model = Model(inputs=model.layers[0].input, outputs=[merged,dsc])

In [33]:
for layer in model.layers:
    if 'flatten' in layer.name:
        break
    else:
        layer.trainable = False

In [35]:
model.get_layer('att').get_weights()

[array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)]

In [1]:
from ResultAnalyser import Result

In [57]:
class Result():
    def __init__(self,log,dann=False,find = False):
        #find is a bool, if true then the log name is searched using the log when the log is only the fold name
        # not the full path . result comparison calls it with the full path. so no need of find
        self.log_dir = '../../Adversarial Heart Sound Results/logs/'
        self.metrics = ['val_macc','val_F1','val_precision','val_sensitivity','val_specificity']
        self.log_name = log.split(' ')[0]
        if(dann):
            self.log_dir = self.log_dir+'dann/'
            self.log_name = self.log_name + " dann"
        if(find):
            self.metrics.append('model_path')
            for x in os.listdir('../../Adversarial Heart Sound Results/logs/'):
                if foldname in x:
                    log = x
                    break
        self.trainer = self.log_name.split('_')[0]
        self.tester = self.log_name.split('_')[1]
        self.df = pd.read_csv(self.log_dir+log+'/training.csv')
        self.df.sort_values(by=['val_macc','val_F1'],ascending=False,inplace = True)
        self.df = dict(self.df.iloc[0][self.metrics])

In [19]:
r = Result(foldname, find = True).df['model_path']
r = r.replace(r[-16:-12],str(int(r[-16:-12])+1).rjust(4,'0'))

In [20]:
r

'../../Adversarial Heart Sound Results/models/bcdefghi_a 2019-10-05 11:26:55.596421/weights.0152-0.5729.hdf5'

In [2]:
domain_list = 'abcdefghi'
test_split = 0
test_domains = 'a'
train_domains = domain_list
for c in test_domains:
    train_domains = train_domains.replace(c,"")
if(test_split==0):
    foldname = train_domains+"_"+test_domains
else:
    foldname = train_domains+"_"+test_domains+"_tune_"+str(test_split)

In [6]:
r 

'../../Adversarial Heart Sound Results/models/bcdefghi_a 2019-10-05 11:26:55.596421/weights.0151-0.5729.hdf5'

In [18]:
r = r.replace(r[-16:-12],str(int(r[-16:-12])+1).rjust(4,'0'))